## **BOSTON 2023**

# Extracción

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Definir la URL para Boston 2023 - repositorio público AWS
url = 'https://s3.amazonaws.com/hubway-data/'

# Se solicita la información a la página y usamos BeautifulSoup para obtenre un listado estructurado en XML y extraemos los archivos .zip de 2023
response = requests.get(url)
files_2023 = [
    file.text for file in BeautifulSoup(response.text, 'xml').find_all('Key')
    if file.text.endswith('.zip') and '2023' in file.text
]

print(f"Encontrados {len(files_2023)} archivos para Boston Blue Bikes 2023")

Encontrados 12 archivos para Boston Blue Bikes 2023


# Transformaciones

La idea es descargar la información mes a mes (solo las columnas necesarias). Se quiere descargar primero la información del mes de enero, escribirlo en un .csv e iterar esta acción para el resto de meses y las columnas necesarias.

In [2]:
# Definimos el nombre del archivo CSV final
output_file = 'Boston_2023.csv'

En este archivo hay una irregularidad a la hora de recoger los datos por parte de Blue Bikes. A partir del mes de mayo los encabezados cambian por lo que, si corremos el código que venimos utilizando con el dataset de 2019 nos dará un error ya que no reconoce el nombre de las columnas a partir de mayor. Por este motivo, se modificará el código para detectar automáticamente si un archivo pertenece al período "enero-abril" o "mayo-diciembre" y asignar los encabezados correctos según corresponda.

### Transformación 1: filtrado de columnas

In [3]:
#T.1: TRANSFORMACIÓN FILTRADO DE COLUMNAS: Recordemos que en este archivo tenemos 2 encabezados distintos. Filtraremos los dos tipos encabezados
columns_early_2023 = ['starttime', 'stoptime', 'start station latitude', 'start station longitude', 'end station latitude', 'end station longitude', 'usertype']
columns_late_2023 = ['started_at', 'ended_at', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']

### Transformación 3: Renombramiento encabezado columnas y transfromación 2: Creación campo ciudad

In [4]:
import os
import zipfile

# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV de enero a abril con las columnas requeridas y columna adicional 'city' (se escogeran las columnas de enero a abril como estándar)
    pd.DataFrame(columns=columns_early_2023 + ['city']).to_csv(output_file, index=False)

# Iteramos sobre cada archivo mensual de 2023 diferenciando si pertenecen a los meses de enero a abril o de mayo a diciembre.
for file_name in files_2023:
    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    nombre_zip = file_name.split('/')[-1] #nos quedamos con el nombre del mes del archivo en lugar de toda la dirección URL

    with open(nombre_zip, "wb") as f: #se crea un archivo vacio para meter la información de la URL de cada mes
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(nombre_zip, "r") as referencia_zip: #se abre el archivo ZIP para leerlo
        referencia_zip.extractall("temp_data") #se extrae la información y lo ponemos en la carpeta temporal

    # Cargar el CSV descomprimido sin especificar columnas para verificar los encabezados
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file))

    # Ahora revisamos las columnas para determinar si corresponde a enero-abril o mayo-diciembre
    if set(columns_early_2023).issubset(df.columns):
        # Si tiene las columnas de enero-abril, selecciona solo las columnas requeridas para estos meses
        df = df[columns_early_2023]
    elif set(columns_late_2023).issubset(df.columns):
        # Si tiene las columnas de mayo-diciembre, selecciona las columnas requeridas
        df = df[columns_late_2023]
        df.columns = ['starttime', 'stoptime', 'start station latitude', 'start station longitude',
                      'end station latitude', 'end station longitude', 'usertype']

    else:
        print(f"Advertencia: Boston 2023 {file_name} no tiene las columnas esperadas.")
        continue  # Saltar archivos que no tienen las columnas esperadas

    # T.2: TRANSFORMACIÓN CREACIÓN CAMPO CIUDAD: Añadir la columna 'city' con el valor "Boston" en todas las filas
    df = df.copy() # se hace una vista porque antes hemos subdividido el dataframe en 2 vistas (early y late), no queremos que afecte al df original
    df['city'] = 'Boston'

    # Reorganizar las columnas para que sigan el orden predeterminado
    df = df[columns_early_2023 + ['city']]

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales
    os.remove(nombre_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con todos los meses de 2023 y la columna 'city' añadida.")

Procesando archivo: 202301-bluebikes-tripdata.zip
Procesando archivo: 202302-bluebikes-tripdata.zip
Procesando archivo: 202303-bluebikes-tripdata.zip
Procesando archivo: 202304-bluebikes-tripdata.zip
Procesando archivo: 202305-bluebikes-tripdata.zip
Procesando archivo: 202306-bluebikes-tripdata.zip
Procesando archivo: 202307-bluebikes-tripdata.zip
Procesando archivo: 202308-bluebikes-tripdata.zip
Procesando archivo: 202309-bluebikes-tripdata.zip
Procesando archivo: 202310-bluebikes-tripdata.zip
Procesando archivo: 202311-bluebikes-tripdata.zip
Procesando archivo: 202312-bluebikes-tripdata.zip
Archivo 'Boston_2023.csv' actualizado con todos los meses de 2023 y la columna 'city' añadida.


In [5]:
# Comprobamos que el csv se ha creado correctamente y que las columnas tengan el formato correcto.
df = pd.read_csv('Boston_2023.csv')
df

# Se puede observar que el formato fecha conserva la información de los milisegundos. Habrá que transformarlo para que tenga el formato estándar elegido (hasta segundos)

,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,usertype,city
0,2023-01-01 00:02:54.0800,2023-01-01 00:07:04.4100,42.349928,-71.077392,42.346520,-71.080658,Subscriber,Boston
1,2023-01-01 00:10:33.3100,2023-01-01 00:18:42.3690,42.349589,-71.079468,42.341356,-71.083370,Customer,Boston
2,2023-01-01 00:10:49.9130,2023-01-01 00:18:31.1280,42.349589,-71.079468,42.341356,-71.083370,Customer,Boston
3,2023-01-01 00:13:58.0640,2023-01-01 00:31:43.5090,42.355536,-71.072869,42.351828,-71.067811,Customer,Boston
4,2023-01-01 00:14:02.6130,2023-01-01 00:24:03.8950,42.351142,-71.073292,42.356683,-71.061666,Subscriber,Boston
...,...,...,...,...,...,...,...,...
3693188,2023-12-20 08:22:55,2023-12-20 08:41:17,42.386748,-71.119019,42.369885,-71.069957,member,Boston
3693189,2023-12-13 08:22:31,2023-12-13 08:39:35,42.386748,-71.119019,42.369885,-71.069957,member,Boston
3693190,2023-12-05 22:10:12,2023-12-05 22:35:39,42.360417,-71.057522,42.355601,-71.103945,casual,Boston
3693191,2023-12-23 12:16:56,2023-12-23 12:41:35,42.386748,-71.119019,42.355601,-71.103945,member,Boston


### Transformación 4: corrección formato fecha

In [6]:
# Reemplaza los milisegundos por vacio, solo cuando están presentes (buscando cualquier dato que tenga un punto seguido de números, esto son los milisegundos)
for col in ['starttime', 'stoptime']:
    df[col] = df[col].str.replace(r'\.\d+$', '', regex=True) #busca en las columnas los puntos (\.) seguidos de números (\d) y selecciona el final de la cadena (+$)

# Verificación rápida
df[['starttime', 'stoptime']].head()

,starttime,stoptime
0,2023-01-01 00:02:54,2023-01-01 00:07:04
1,2023-01-01 00:10:33,2023-01-01 00:18:42
2,2023-01-01 00:10:49,2023-01-01 00:18:31
3,2023-01-01 00:13:58,2023-01-01 00:31:43
4,2023-01-01 00:14:02,2023-01-01 00:24:03


### Transformación 5: Corrección formato tipo de cliente

Por último, también existe una inconsistencia en el formato del tipo de usuario. En los primeros meses del año los tipos de usuarios se desglosan en "Subscriber y Customer" mientras que, durante los últimos meses se desglosan en "member" y "casual". Por este motivo, es necesario homogeneizar la información de esta columna llevando a cabo otra transformación.

In [7]:
df['usertype'] = df['usertype'].replace({'member': 'Subscriber', 'casual': 'Customer'}) # Se sustituyen los nombres de los tipos de usuarios para homogeneizarlos
# Verificación rápida
df[['usertype']].head()

,usertype
0,Subscriber
1,Customer
2,Customer
3,Customer
4,Subscriber


In [8]:
#Descargar en csv tras las transformaciones pertinentes:
print(f"El archivo Boston 2023 {df.shape[0]} filas y {df.shape[1]} columnas")
df.to_csv('Boston_2023.csv', index=False)

El archivo Boston 2023 3693193 filas y 8 columnas
